In [ ]:
# # Clone xvr repo and move into the directory
# !git clone https://github.com/eigenvivek/xvr.git
# %cd xvr

# # Install PyTorch for CUDA 11.8 (Colab T4 GPU)
# !pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# # Install xvr and dependencies
# !pip install git+https://github.com/eigenvivek/xvr.git
!pip install wandb nibabel torchio tqdm


In [ ]:
import torch
import torchvision
import torchaudio
import nibabel as nib
import torchio
import tqdm
import wandb

# Check CUDA availability and GPU info
print("Torch CUDA available:", torch.cuda.is_available())
print("Torch version:", torch.__version__)
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("CUDA Version:", torch.version.cuda)

# Check xvr import
try:
    import xvr
    print("✅ xvr imported successfully")
except ImportError as e:
    print("❌ xvr import failed:", e)


In [ ]:
import os

# Set a value
os.environ["WANDB_API_KEY"] = "524773117955f64051f5e2bd2ecef2642a677a34"

In [ ]:
# Fix wandb login inside xvr training script (do this only once)
!sed -i 's/wandb.login(key=os.environ\["WANDB_API_KEY"\])/wandb.login()/' /usr/local/lib/python3.*/dist-packages/xvr/commands/train.py

In [ ]:
import sys
import os

# Add the path to xvr/src so you can import xvr.*
sys.path.append(os.path.join(os.getcwd(), 'xvr', 'src'))


In [ ]:
import sys
import os

# Step 1: Add xvr/src to Python path so imports work
sys.path.append(os.path.join(os.getcwd(), 'xvr', 'src'))

# Step 2: Patch wandb.login line in local train.py
train_py = os.path.join(os.getcwd(), 'xvr', 'src', 'xvr', 'commands', 'train.py')

if os.path.exists(train_py):
    with open(train_py, 'r') as f:
        lines = f.readlines()

    with open(train_py, 'w') as f:
        for line in lines:
            f.write(line.replace(
                'wandb.login(key=os.environ["WANDB_API_KEY"])',
                'wandb.login()'
            ))

    print(f"✅ Patched wandb.login() in: {train_py}")
else:
    print("❌ train.py not found. Check the path.")


In [ ]:
# !xvr train \
#   --inpath "SE000002.nii" \
#   --outpath "output" \
#   --r1 -45 45 --r2 -45 45 --r3 -15 15 \
#   --tx -200 200 --ty -450 -1000 --tz -200 200 \
#   --sdd 1000.0 \
#   --height 128 \
#   --delx 0.316 \
#   --batch_size 116 \
#   --n_epochs 1000 \
#   --n_batches_per_epoch 100

In [ ]:
import xvr
!xvr --help

In [ ]:
import subprocess

# Updated Windows-style absolute paths
input_path = "SE000002.nii.gz"
output_path = "output"

command = [
    "xvr", "train",
    "--inpath", input_path,
    "--outpath", output_path,
    "--r1", "-45", "45",
    "--r2", "-45", "45",
    "--r3", "-45", "45",
    "--tx", "-200", "200",
    "--ty", "-450", "-1000",
    "--tz", "-200", "200",
    "--sdd", "1000.00",
    "--height", "128",
    "--delx", "2.42",
    "--lr", "0.01",
    "--reverse_x_axis",
    "--orientation", "[PA]",
    "--batch_size", "116",
    "--n_epochs", "1000",
    "--n_batches_per_epoch", "100",
    "--project", "RXVR1.2"
]

# Run the command and print output live
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in process.stdout:
    print(line, end='')  # print output as it streams


In [ ]:
# process.kill()
# print("✅ Training process terminated.")

# import torch
# import gc

# # Garbage collect unreferenced objects
# gc.collect()

# # Clear unused memory from PyTorch's CUDA allocator
# torch.cuda.empty_cache()

# print("✅ GPU cache cleared.")


NameError: name 'process' is not defined

In [ ]:
# !xvr train --help

In [ ]:
# !xvr animate \
#   --inpath "/content/drive/MyDrive/SE000002.nii" \
#   --checkpoint "/content/drive/MyDrive/Trained Registration model/xvr_0125.pth" \
#   --outpath "/content/drive/MyDrive/Trained Registration model" \
#   --height 128 --delx 0.5